### Behind the pipeline

In [44]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=6, linewidth=75, sci_mode=False)

让我们从一个完整的例子开始，看看当我们在第1章中执行以下代码时，在幕后发生了什么:

In [46]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
])
result

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

让我们快速浏览一下这些。

### Preprocessing with a tokenizer

与其他神经网络一样，Transformer模型并不能直接处理原始文本，因此pipeline的第一步是将文本输入转换为模型可以理解的数字。为此，我们使用一个tokenizer，它将负责:
- 将输入拆分为称为标记的单词、子单词或符号(如标点符号)
- 将每个标记映射为一个数字
- 添加可能对模型有用的额外输入

所有这些预处理都需要以与模型预训练时完全相同的方式进行，因此首先需要从Model Hub下载该信息。为此，我们使用AutoTokenizer类及其from_pretrained()方法。使用模型的checkpoint名称，它将自动获取与模型的tokenizer相关的数据并缓存它(其仅在首次运行时下载)。

这里sentiment-analysis管线默认的checkpoint为 distilbert-base-uncased-finetuned-sst-2-english，因此可以运行如下代码：

In [7]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

一旦有了tokenizer，就可以直接将句子传递给它，这样就得到了一个可以提供给模型的字典内容。接下来唯一要做的就是将输入id列表转换为tensor。
使用Transformer时，不必担心使用哪个ML(机器学习)框架作为后端。它可能是PyTorch或TensorFlow，或某些模型的Flex。然而，Transformer模型只接受张量作为输入。如果这是你第一次听说张量，你可以把它们想象成NumPy数组。NumPy数组可以是标量(0D)，矢量(1D)，矩阵(2D)，或者具有更多维度。它实际上是一个张量。与其他ML框架的张量的行为类似，并且通常像NumPy数组一样简单地实例化。
指定我们想要返回的张量类型(PyTorch, TensorFlow或普通NumPy)，可以使用return_tensors参数:

In [9]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [12]:
inputs, inputs["input_ids"].shape, inputs["attention_mask"].shape

({'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
          12172,  2607,  2026,  2878,  2166,  1012,   102],
         [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
              0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])},
 torch.Size([2, 16]),
 torch.Size([2, 16]))

这里先不用关心参数padding和truncation，稍后会做解释。这里只需记住：可以传递单句或列表，以及指定想要返回的tensor类型(如果没有传递类型，将得到一个列表的列表作为结果)。
输出本身是一个包含两个键input_ids和attention_mask的字典。input_ids包含两行整数(每个句子一行)，它们是每个句子中token的唯一标识符。我们将在本章后面解释什么是attention_mask。

### Going through the model

我们可以像下载tokenizer一样下载预训练模型。Transformers提供了一个AutoModel类，其也有from_pretrained()方法:

In [13]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


在代码中，我们下载了之前在pipeline中使用的相同checkpoint(实际上它应该已经被缓存了)，并用它实例化了一个模型。该框架仅包含基本的Transformer模块: 给定一些输入，其输出hidden states(也称为features特征)的内容。对于每个模型输入，我们将检索一个高维向量，表示Transformer模型对该输入的上下文理解。
如果不明白这些，不要担心，我们稍后会再解释。
这些hidden states本身可能很有用，其通常是模型的另一部分输入，被称为head。在第一章中，不同的任务可以用相同的框架来执行，但是每个任务将有一个不同的头与之相关联。


#### A high-dimensional vector?

Transformer输出的vector通常都比较大，一般有三个维度：
- Batch size: 一次处理的序列数量(在我们的示例中为2)。
- Sequence length: 数字化表示的序列长度，一般值tensor的shape大小(在我们的示例中为16)
- Hidden size: 模型输入的vector尺寸

之所以被称为“高维”是因为最后一个值Hidden size，其值可以非常大(较小的模型通常为768，而较大的模型可以达到3072或更多)。
如果我们将预处理后的输入输入到模型中，我们可以看到这一点:

In [14]:
outputs = model(**inputs)

In [18]:
outputs.last_hidden_state.shape

torch.Size([2, 16, 768])

注意，Transformer的输出类似于namedtuples或字典。可以通过属性(像我们所做的那样)或键(outputs["last_hidden_state"])访问元素，如果确切地知道要查找的东西在哪里(outputs[0])，甚至可以通过索引访问元素。

#### Model heads: Making sense out of numbers

模型头部将隐藏状态的高维向量作为输入，并将它们投射到不同的维度上。它们通常由一个或几个线性层组成。
Transformer模型的输出直接发送到模型头进行处理。
在Transformer中有许多不同的架构，每个架构都是围绕解决特定任务而设计的。以下是一份不详尽的清单:
- Model (retrieve the hidden states)
- ForCausalLM
- ForMaskedLM
- ForMultipleChoice
- ForQuestionAnswering
- ForSequenceClassification
- ForTokenClassification
and others 🤗

对于我们的示例，我们需要一个具有序列分类头的模型(能够将句子分类为肯定的或否定的)。所以，我们实际上不会使用AutoModel类，AutoModelForSequenceClassification:

In [34]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [35]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.56,  1.61],
        [ 4.17, -3.35]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

现在，如果我们看一下输出的shape，维度将会低得多: 模型头部将我们之前看到的高维向量作为输入，输出向量包含两个值(每个标签一个):
torch.Size([2, 2])。
因为我们只有两个句子和两个标签，所以我们从模型中得到的结果是形状为2 x 2的。

### Postprocessing the output

我们从模型中得到的输出值本身并不直观。让我们一起来看看:

In [36]:
outputs.logits

tensor([[-1.56,  1.61],
        [ 4.17, -3.35]], grad_fn=<AddmmBackward0>)

我们的模型预测第一句为[-1.5607,1.6123]，第二句为[4.1692，-3.3464]。这些不是概率，而是logits，即模型最后一层输出的原始的、未标准化的分数。要转换为概率，它们需要经过一个SoftMax层(所有Transformer模型都会输出logits，因为用于训练的损失函数通常会将最后一个激活函数(如SoftMax)与实际损失函数(如交叉熵)融合在一起):

In [45]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions

tensor([[    0.040195,     0.959805],
        [    0.999456,     0.000544]], grad_fn=<SoftmaxBackward0>)

现在我们可以看到，模型对第一句的预测结果为[0.0402,0.9598]，对第二句的预测结果为[0.9995,0.0005]，这些是可识别的概率。
为了获得每个位置对应的标签，我们可以检查模型配置的id2label属性(下一节将详细介绍):

In [41]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

现在我们可以得出结论，该模型预测如下:
- 第一句: NEGATIVE 0.0402, POSITIVE 0.9598
- 第二句: NEGATIVE 0.9995, POSITIVE 0.0005

我们已经成功地复现了pipeline的三个步骤: 使用标记器进行预处理，通过模型传递输入，以及后处理!

### 完整示例代码

In [55]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions

tensor([[    0.040195,     0.959805],
        [    0.999456,     0.000544]], grad_fn=<SoftmaxBackward0>)

In [56]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}